# Getting Started

In this tutorial, you will know how to
- use the models in **ConvLab-3** to build a dialog agent.

Let's get started!

## Environment setup
Run the command below to install ConvLab-3. Then restart the notebook and skip this commend.

In [ ]:
# first install ConvLab-3 and restart the notebook
! git clone --depth 1 https://github.com/ConvLab/ConvLab-3.git && cd ConvLab-3 && pip install -e .

## build an agent

We use the models adapted on [Multiwoz](https://www.aclweb.org/anthology/D18-1547)  dataset to build our agent. This pipeline agent consists of NLU, DST, Policy and NLG modules.

First, import some models:

In [ ]:
# common import: convlab.$module.$model.$dataset
from convlab.base_models.t5.nlu import T5NLU
from convlab.base_models.t5.dst import T5DST
from convlab.base_models.t5.nlg import T5NLG
from convlab.policy.vector.vector_nodes import VectorNodes
from convlab.policy.vtrace_DPT import VTRACE
from convlab.dialog_agent import PipelineAgent, BiSession
from convlab.evaluator.multiwoz_eval import MultiWozEvaluator
from pprint import pprint
import random
import numpy as np
import torch

Then, create the models and build an agent on Multiwoz 2.1 dataset:

In [ ]:
# go to README.md of each model for more information
sys_nlu = T5NLU(speaker='user', context_window_size=0, model_name_or_path='ConvLab/t5-small-nlu-multiwoz21')
sys_dst = T5DST(dataset_name='multiwoz21', speaker='user', context_window_size=100, model_name_or_path='ConvLab/t5-small-dst-multiwoz21')
# Download pre-trained DDPT model
! wget https://huggingface.co/ConvLab/ddpt-policy-multiwoz21/resolve/main/supervised.pol.mdl --directory-prefix="convlab/policy/vtrace_DPT"
vectorizer = VectorNodes(dataset_name='multiwoz21',
                         use_masking=True,
                         manually_add_entity_names=True,
                         seed=0,
                         filter_state=True)
sys_policy = VTRACE(is_train=False,
              seed=0,
              vectorizer=vectorizer,
              load_path="convlab/policy/vtrace_DPT/supervised")
sys_nlg = T5NLG(speaker='system', context_window_size=0, model_name_or_path='ConvLab/t5-small-nlg-multiwoz21')
# assemble
sys_agent = PipelineAgent(sys_nlu, sys_dst, sys_policy, sys_nlg, name='sys')

That's all! Let's chat with the agent using its response function:

In [ ]:
sys_agent.init_session()
sys_agent.response("I want to find a hotel in the expensive pricerange")

In [ ]:
sys_agent.response("Which type of hotel is it ?")

In [ ]:
sys_agent.response("OK , where is its address ?")

In [ ]:
sys_agent.response("Thank you !")

In [ ]:
sys_agent.response("Try to find me a Chinese restaurant in south area .")

In [ ]:
sys_agent.response("Which kind of food it provides ?")

In [ ]:
sys_agent.response("Book a table for 5 , this Sunday .")